In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
#Read the ods File
path = "../Raw Data/feb19.ods"
df = pd.read_excel(path, engine="odf")
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,11797,84.26%,4271,50389369,57980978,117971425
1,TheGrandHotel,Taipei,0,0,NaN,0,0,0,0
2,國賓⼤飯店,Taipei,422,8318,70.40%,3460,28776666,86621613,123207387
3,TheAmbassadorHotel-Taipei,Taipei,0,0,NaN,0,0,0,0
4,台北華國⼤飯店,Taipei,326,7293,79.90%,2673,19490615,11199940,34808587


In [3]:
#Clean Data 
df = pd.DataFrame(df, columns=['Hotel','Area','Room','Demand','Occupancy','ADR','Room_Revenue','F&B_Revenue','Total_Revenue'])
df.head()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,11797,84.26%,4271,50389369,57980978,117971425
1,TheGrandHotel,Taipei,0,0,NaN,0,0,0,0
2,國賓⼤飯店,Taipei,422,8318,70.40%,3460,28776666,86621613,123207387
3,TheAmbassadorHotel-Taipei,Taipei,0,0,NaN,0,0,0,0
4,台北華國⼤飯店,Taipei,326,7293,79.90%,2673,19490615,11199940,34808587


In [4]:
df = df.dropna()
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
0,圓⼭⼤飯店,Taipei,500,11797,84.26%,4271,50389369,57980978,117971425
2,國賓⼤飯店,Taipei,422,8318,70.40%,3460,28776666,86621613,123207387
4,台北華國⼤飯店,Taipei,326,7293,79.90%,2673,19490615,11199940,34808587
6,華泰王⼦⼤飯店,Taipei,213,4032,67.61%,2708,10918928,13295360,27567770
8,國王⼤飯店,Taipei,97,1902,70.03%,1498,2848304,808237,3742937
...,...,...,...,...,...,...,...,...,...
244,台東桂⽥喜來登酒店,Others,290,3387,41.71%,4718,15980744,10835712,27651014
246,寶華⼤飯店,Others,78,2,0.09%,3650,7300,0,9300
248,⻑榮桂冠酒店(基隆),Others,140,2545,64.92%,3547,9027928,5806689,17990586
250,鈺通⼤飯店,Others,120,1474,43.87%,2112,3113620,5221772,8378980


In [5]:
df.loc[df['Hotel']=='華泰王⼦⼤飯店']

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue
6,華泰王⼦⼤飯店,Taipei,213,4032,67.61%,2708,10918928,13295360,27567770


In [6]:
print(df.dtypes)

Hotel            object
Area             object
Room              int64
Demand            int64
Occupancy        object
ADR               int64
Room_Revenue      int64
F&B_Revenue       int64
Total_Revenue     int64
dtype: object


df['Demand'] = df['Demand'].replace(',', '', regex=True)
df['ADR'] = df['ADR'].replace(',', '', regex=True)
df['Room_Revenue'] = df['Room_Revenue'].replace(',', '', regex=True)
df['F&B_Revenue'] = df['F&B_Revenue'].replace(',', '', regex=True)
df['Total_Revenue'] = df['Total_Revenue'].replace(',', '', regex=True)

In [7]:
df['Occupancy'] = df['Occupancy'].str.rstrip('%').astype('float') / 100.0

In [8]:
df[df['Occupancy'].isnull()]

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue


In [9]:
df.astype({'Room': 'int64',
          'Demand': 'int64',
          'Occupancy': 'int64',
          'ADR': 'float64',
          'Room_Revenue': 'int64',
          'F&B_Revenue': 'int64',
          'Total_Revenue': 'int64',
          }).dtypes

Hotel             object
Area              object
Room               int64
Demand             int64
Occupancy          int64
ADR              float64
Room_Revenue       int64
F&B_Revenue        int64
Total_Revenue      int64
dtype: object

In [10]:
Day = 28

In [11]:
#Calculate/Add RevPAR and Demand
df['RevPAR'] = df['Occupancy']*df['ADR']
df['Supply'] = df['Room']*Day
df

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
0,圓⼭⼤飯店,Taipei,500,11797,0.8426,4271,50389369,57980978,117971425,3598.7446,14000
2,國賓⼤飯店,Taipei,422,8318,0.7040,3460,28776666,86621613,123207387,2435.8400,11816
4,台北華國⼤飯店,Taipei,326,7293,0.7990,2673,19490615,11199940,34808587,2135.7270,9128
6,華泰王⼦⼤飯店,Taipei,213,4032,0.6761,2708,10918928,13295360,27567770,1830.8788,5964
8,國王⼤飯店,Taipei,97,1902,0.7003,1498,2848304,808237,3742937,1049.0494,2716
...,...,...,...,...,...,...,...,...,...,...,...
244,台東桂⽥喜來登酒店,Others,290,3387,0.4171,4718,15980744,10835712,27651014,1967.8778,8120
246,寶華⼤飯店,Others,78,2,0.0009,3650,7300,0,9300,3.2850,2184
248,⻑榮桂冠酒店(基隆),Others,140,2545,0.6492,3547,9027928,5806689,17990586,2302.7124,3920
250,鈺通⼤飯店,Others,120,1474,0.4387,2112,3113620,5221772,8378980,926.5344,3360


In [12]:
#Calculate Monthly Room/Supply/Demand/Room Revenue/F&B Revenue/Total Revenue
date = "2019-02-28"
room = round(df["Room"].sum(),0)
supply = round(df["Supply"].sum(),0)
demand = round(df['Demand'].sum(),0)
room_revenue = df["Room_Revenue"].sum()
fb_revenue = df["F&B_Revenue"].sum()
total_revenue = df["Total_Revenue"].sum()

In [13]:
#Calculate Monthly Occupancy/ADR/RevPAR
occ = round((demand/supply),2)
adr = round((room_revenue/demand), 2)
#revpar = round((occ*adr), 2)
revpar = round((room_revenue/supply), 2)

In [14]:
#Append "Total" Row
df = df.append([{'Hotel': 'Total',
                'Area': 'Total',
                'Room': room,
                'Supply': supply,
                'Occupancy': occ,
                'ADR': adr,
                'Room_Revenue': room_revenue,
                'F&B_Revenue': fb_revenue,
                'Total_Revenue': total_revenue,
                'RevPAR': revpar,
                'Demand': demand}], ignore_index=True)
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply
123,寶華⼤飯店,Others,78,2,0.0009,3650.00,7300,0,9300,3.2850,2184
124,⻑榮桂冠酒店(基隆),Others,140,2545,0.6492,3547.00,9027928,5806689,17990586,2302.7124,3920
125,鈺通⼤飯店,Others,120,1474,0.4387,2112.00,3113620,5221772,8378980,926.5344,3360
126,陸島酒店,Others,47,254,0.1930,2012.00,510969,42400,598834,388.3160,1316
127,Total,Total,29420,525104,0.6400,4046.63,2124900522,2202112505,4882778853,2579.5100,823760


In [15]:
#Add "Date" Column 
df['Date'] = date
df.tail()

,Hotel,Area,Room,Demand,Occupancy,ADR,Room_Revenue,F&B_Revenue,Total_Revenue,RevPAR,Supply,Date
123,寶華⼤飯店,Others,78,2,0.0009,3650.00,7300,0,9300,3.2850,2184,2019-02-28
124,⻑榮桂冠酒店(基隆),Others,140,2545,0.6492,3547.00,9027928,5806689,17990586,2302.7124,3920,2019-02-28
125,鈺通⼤飯店,Others,120,1474,0.4387,2112.00,3113620,5221772,8378980,926.5344,3360,2019-02-28
126,陸島酒店,Others,47,254,0.1930,2012.00,510969,42400,598834,388.3160,1316,2019-02-28
127,Total,Total,29420,525104,0.6400,4046.63,2124900522,2202112505,4882778853,2579.5100,823760,2019-02-28


In [16]:
df['Date'] = pd.to_datetime(df['Date']).dt.to_period('D')

In [17]:
#Format 'Summary'
df['Date'] = df['Date'].dt.to_timestamp()
#df = df.style.format({'occ':'{:.0%}',
                    #'demand':'{:.0f}',
                    #'adr':'{:.2f}',
                    #'revpar':'{:.2f}'})

In [18]:
print(df.dtypes)

Hotel                    object
Area                     object
Room                      int64
Demand                    int64
Occupancy               float64
ADR                     float64
Room_Revenue              int64
F&B_Revenue               int64
Total_Revenue             int64
RevPAR                  float64
Supply                    int64
Date             datetime64[ns]
dtype: object


#Create DataFrame 'Summary'
summary = pd.DataFrame({
    'date':date,
    'room':room,
    'supply':supply,
    'demand':demand,
    'room_revenue': room_revenue, 
    'fb_revenue': fb_revenue,
    'total_revenue': total_revenue,
    'occ': occ,
    'adr': adr,
    'revpar':revpar   
}, index=[0])
summary

#Format Summary
summary['date'] = pd.to_datetime(summary['date']).dt.to_period('M')
sep20 = summary.style.format({'occ':'{:.0%}',
                    'demand':'{:.0f}',
                    'adr':'{:.2f}',
                    'revpar':'{:.2f}'})

sep20

df.to_excel(r'static\data\2020\sep20.xlsx', index = False, encoding='utf-8-sig')

In [19]:
df.to_csv(r'..\static\data\2019\feb19.csv', index = False, encoding='utf-8-sig')

In [20]:
from sqlalchemy import create_engine

In [21]:
connection_string = "postgres:postgres@localhost:5432/hotel_db"
engine = create_engine(f'postgresql://{connection_string}', encoding = 'utf-8')

In [22]:
engine.table_names()

['jan20',
 'oct20',
 'nov20',
 'dec20',
 'combine20',
 'dec19',
 'nov19',
 'oct19',
 'sep19',
 'jun19',
 'aug19',
 'jul19',
 'may19',
 'apr19',
 'mar19',
 'feb19',
 'jan19',
 'feb20',
 'mar20',
 'apr20',
 'may20',
 'jun20',
 'jul20',
 'aug20',
 'sep20']

In [23]:
df.to_sql(name='feb19', con=engine, if_exists='append', index=False)